In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import time

In [2]:
url = 'https://hh.ru/search/vacancy'

headers = {'User-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36'}

params = {'area': 1, 
          'ored_clusters': True, 
          'text': '', 
          'search_period': 0, 
          'page': 0}

session = requests.Session()

In [3]:
job = str(input('Введите должность: '))
#страницы сайта я буду сохранять в список
pages = []
items = 'not_empty'

#делаем запросы до тех пор, пока на страницах есть вакансии
while items:
    params['text'] = job
    response = session.get(url, headers=headers, params=params)
    soup = bs(response.text, 'lxml')
    #проверяем есть ли вакансии на полученной странице
    items = soup.find_all(attrs={'class':'serp-item'}) 
    pages.append(soup)
    params['page'] += 1
    #делаем небольшую паузу между запросами, чтобы не ддосить сайт
    time.sleep(1)

Введите должность: Data scientist


In [4]:
#информация о вакансиях будет сохраняться в словарь, из которого потом получим датафрейм
items_dict = {'name':[], 
              'min_salary':[], 
              'max_salary':[], 
              'currency':[], 
              'link':[], 
              'source':[], 
              'employer':[], 
              'location':[]}

for page in pages:
    #поиск всех вакансий
    items = page.find_all(attrs={'class':'serp-item'})  

    #далее ищем нужную информацию по каждой вакансии и добавляем её в словарь
    for i in items:
        items_dict['name'].append(i.find(attrs={'class':'serp-item__title'}).text)
        #в try-except здесь обернут процесс получения информации о зарплате. Не у всех вакансий указана зарплата, 
        #поэтому в многих случаях здесь возникает ошибка
        try:
            temp_list = i.find(attrs={'data-qa':'vacancy-serp__vacancy-compensation'}).text.replace('\u202f','').replace('–','').split()
            if 'от' in temp_list:
                items_dict['min_salary'].append(temp_list[1])
                items_dict['max_salary'].append(None)
                items_dict['currency'].append(temp_list[2])
            elif 'до' in temp_list:
                items_dict['min_salary'].append(None)
                items_dict['max_salary'].append(temp_list[1])
                items_dict['currency'].append(temp_list[2])
            else:
                items_dict['min_salary'].append(temp_list[0])
                items_dict['max_salary'].append(temp_list[1])
                items_dict['currency'].append(temp_list[2])
        except:
            items_dict['min_salary'].append(None)
            items_dict['max_salary'].append(None)
            items_dict['currency'].append(None)
        items_dict['link'].append(i.find(attrs={'data-qa':'serp-item__title'})['href'].split('?')[0])
        items_dict['source'].append('hh.ru')
        items_dict['employer'].append(i.find(attrs={'data-qa':'vacancy-serp__vacancy-employer'}).text)
        items_dict['location'].append(i.find(attrs={'data-qa':'vacancy-serp__vacancy-address'}).text)


In [5]:
result = pd.DataFrame(data=items_dict)
result

,name,min_salary,max_salary,currency,link,source,employer,location
0,Senior data scientist в команду StatWars,None,None,None,https://hh.ru/vacancy/81720032,hh.ru,ООО HeadHunter::Analytics/Data Science,"Москва, Алексеевская"
1,Data Scientist в b2c направление,None,None,None,https://hh.ru/vacancy/81647441,hh.ru,ООО HeadHunter::Analytics/Data Science,"Москва, Алексеевская"
2,Chief Data Scientist,None,None,None,https://hh.ru/vacancy/79057338,hh.ru,Сбер. Data Science,Москва
3,ML engineer / Data engineer,None,None,None,https://hh.ru/vacancy/81402430,hh.ru,Сбер для экспертов,Москва
4,Data Scientist,250000,350000,руб.,https://hh.ru/vacancy/80412928,hh.ru,Aimers,Москва
...,...,...,...,...,...,...,...,...
117,Data Engineer в команду Operational DWH,None,None,None,https://hh.ru/vacancy/81576115,hh.ru,Райффайзен Банк,Москва
118,Директор центра компетенций по персонализирова...,None,None,None,https://hh.ru/vacancy/79673006,hh.ru,ООО РСХБ-Интех,Москва
119,ModelOps specialist,None,None,None,https://hh.ru/vacancy/81564805,hh.ru,ООО GlowByte,Москва
120,Портфельный риск-аналитик / Middle Data Scient...,None,None,None,https://hh.ru/vacancy/81033363,hh.ru,ООО Элемент Лизинг,"Москва, Кутузовская и еще 1"


In [6]:
result.to_csv('data.csv', index=False)